In [1]:
from graphviz import Digraph
import os

### push operators and ( only to stack first then pop them to output by precedence at last after the alphnum goes to output directly first when readed 

In [2]:
# This part defines operator priority and checks if a character is an operator.
def precedence(op):
    if op == '*':
        return 3
    elif op == '.':
        return 2
    elif op == '+':
        return 1
    return 0
#is_operator('*') → True
#is_operator('a') → False
def is_operator(c):
    return c in {'+', '*', '.','^+'}  # distinguish between operators and regular letters/numbers.

def to_postfix(regex): # convers infix to postifx notation by using shunting-yard algorithm which is easier to process when building the NFA.
    def insert_concat(regex):
        result = "" # Creates an empty string result to build the modified regex.
        for i in range(len(regex)): # Iterates over each character in the input regex using index i.
            c1 = regex[i] #Stores the current character in c1. # Current character (e.g., for "ab": i=0 → 'a'; i=1 → 'b')
            result += c1 
            if i + 1 < len(regex): #Ensures we don’t go out of bounds when checking the next character by (c2).
                c2 = regex[i + 1] # Next character (e.g., for "ab": i=0 → 'b'; i=1 → not checked)
                if (c1.isalnum() or c1 == '*' or c1 == ')') and (c2.isalnum() or c2 == '('):
            # Insert '.' between:
            # 1. Two letters (e.g., 'ab' → 'a.b')
            # 2. After '*' or ')' (e.g., 'a*b' → 'a*.b')
            # 3. Before '(' (e.g., 'a(b' → 'a.(b')
                    result += '.'
        return result

    regex = insert_concat(regex) # Calls the insert_concat function to modify the regex by adding '.' where necessary.

    stack = []  # Temporarily holds operators and ( during conversion.
    output = "" # Stores the final postfix expression.
    for char in regex: # Iterates over every character in the modified regex (e.g., a.(b|c)*.d)
        if char.isalnum(): # Letters/numbers go straight to output
            output += char # (e.g., for "a.b.c": output = "abc") 
        elif char == '(':  
            stack.append(char) 
        elif char == ')': 
            while stack and stack[-1] != '(': # while not on the top of the stack the opening parentheses start to pop all operators in the stack and put them in the output, untill you reach to its lowest which is the open parentheses due to is the first thing goes in the stack 
                output += stack.pop() 
            if not stack: # If stack is empty, it means there was no matching '(' for this ')'
                raise ValueError("Mismatched parentheses")
            stack.pop()  # Pop the '(' from the stack but do not add it to output.
        elif is_operator(char): # Operators are handled based on precedence.
            while stack and stack[-1] != '(' and precedence(char) <= precedence(stack[-1]): #While the top of stack has a higher-precedence operator, pop it to output
                output += stack.pop()  # For "a+b*":'*' has higher precedence than '+', so '*' is popped first → output = "ab*+".
            stack.append(char) # Push the current operator to stack.
        else:
            raise ValueError(f"Invalid character in regex: {char}") # like inpute # or $ or % etc.

    while stack: #This loop ensures all remaining operators are popped from the stack and checks for mismatched parentheses before returning the final postfix expression.
        top = stack.pop()
        if top == '(': # If the popped item is (, it means there was no closing ) in the regex.
            raise ValueError("Mismatched parentheses")
        output += top # else Appends the popped operator (+, *, etc.) to output.

    return output

In [3]:
class State: # Represents a node in the NFA.
    def __init__(self):
        self.transitions = {}  # Key: Input symbol (e.g., 'a', 'e' for ε). Value: Set of next states. #for example, {'a': {state1, state2}} means on input 'a', it can go to state1 or state2.
        self.id = None         #for example start state id equal to 1 so that it can be used to identify the state in the NFA and no another state can have the same that id. 

class NFA:
    def __init__(self, start, end):
        self.start = start
        self.end = end

def build_nfa(postfix): # Builds an NFA from the postfix regex using a stack-based approach.
    state_id = 0 # Global variable to assign unique IDs to states.
    def new_state(): # Creates a new state with a unique ID.
        nonlocal state_id # Allows access to the state_id variable defined in the outer scope.
        s = State() # Creates a new instance of the State class.
        s.id = state_id # Assigns the current state_id to the new state.
        state_id += 1 # Increments the state_id for the next state.
        return s 

    stack = [] # Stack to hold NFA components during construction. for example, if the postfix is "ab.c*", the stack will hold NFA components for 'a', 'b', and 'c*' as they are processed.

    for char in postfix: # Iterates over each character in the postfix regex.  ab.c*| which represents the regex (a·b)|c*.
        if char.isalnum(): #if the char for example is 'a' is alphanumeric
            start = new_state() # State 0
            end = new_state()   # State 1
            start.transitions[char] = {end} # 0 --a--> 1
            stack.append(NFA(start, end)) #Push NFA(0-->1) to stack
        elif char == '*': # If the character is '*'
            nfa1 = stack.pop() # pop the top NFA from the stack and create a new NFA for the Kleene star operation. (4-->5)
            start = new_state() # creates a new start state for the Kleene star NFA.
            end = new_state() # creates a new end state for the Kleene star NFA.
            start.transitions['e'] = {nfa1.start, end} # From new start (6) to old start (4) and new end (7)
            nfa1.end.transitions['e'] = {nfa1.start, end} # From old end (5) back to old start (4) and to new end (7)
            stack.append(NFA(start, end))
        elif char == '.':
            nfa2 = stack.pop() # NFA(2→3) 
            nfa1 = stack.pop() # NFA(0→1)
            nfa1.end.transitions['e'] = {nfa2.start} # Connects the end state of nfa1 to the start state of nfa2 using ε-transition. # 1 --ε--> 2
            stack.append(NFA(nfa1.start, nfa2.end)) #NFA(0-->3) (0 --a--> 1 --ε--> 2 --b--> 3)
        elif char == '+':
            nfa2 = stack.pop()  # NFA(6→7) for c*
            nfa1 = stack.pop()  # NFA(0→3) for ab
            start = new_state() # State 8
            end = new_state()   # State 9
            start.transitions['e'] = {nfa1.start, nfa2.start} # 8 → 0 and 8 → 6
            nfa1.end.transitions['e'] = {end}  # 3 → 9 
            nfa2.end.transitions['e'] = {end}  # 7 → 9
            stack.append(NFA(start, end))   # Push the new NFA (8→9) to the stack.
        else:
            raise ValueError(f"Unknown character in postfix: {char}")

    return stack.pop()

In [4]:
def print_transition_table(nfa):
    visited = set()
    def dfs(state):
        if state.id in visited:
            return []
        visited.add(state.id)
        rows = []
        for symbol, next_states in state.transitions.items():
            for next_state in next_states:
                rows.append((state.id, symbol, next_state.id))
                rows.extend(dfs(next_state))
        return rows

    transitions = dfs(nfa.start)
    print("Transition table:")
    print("_______________")
    print("| from state    | input | to states")
    for t in transitions:
        print(f"| {t[0]:<13} | {t[1]:<5} | {t[2]}")
    print("_______________")
    print(f"Start state: {nfa.start.id}")
    print(f"End state: {nfa.end.id}")
    return transitions, nfa.start.id, nfa.end.id

def visualize_nfa(transitions, start_state, end_state, filename="nfa"):
    dot = Digraph()
    dot.attr(rankdir='LR')

    # Invisible start arrow
    dot.node('start', shape='point')
    dot.edge('start', str(start_state))

    states = set()
    for from_state, symbol, to_state in transitions:
        states.add(from_state)
        states.add(to_state)
        label = 'ε' if symbol == 'e' else symbol
        dot.edge(str(from_state), str(to_state), label=label)

    for s in states:
        if s == end_state:
            dot.node(str(s), shape='doublecircle')
        else:
            dot.node(str(s), shape='circle')

    dot.render(filename, view=True)

In [5]:
def main():
    regex = input("Enter regular expression: ")
    postfix = to_postfix(regex)
    print("Postfix expression:", postfix)
    nfa = build_nfa(postfix)
    transitions, start_state, end_state = print_transition_table(nfa)
    visualize_nfa(transitions, start_state, end_state)

In [9]:
if __name__ == "__main__":
    main()

Postfix expression: ab.c+*
Transition table:
_______________
| from state    | input | to states
| 8             | e     | 6
| 6             | e     | 0
| 0             | a     | 1
| 1             | e     | 2
| 2             | b     | 3
| 3             | e     | 7
| 7             | e     | 6
| 7             | e     | 9
| 6             | e     | 4
| 4             | c     | 5
| 5             | e     | 7
| 8             | e     | 9
_______________
Start state: 8
End state: 9
